In [104]:
import pandas as pd
import matplotlib.pyplot as plt
import difflib

%matplotlib inline

In [15]:
# files = ['turnstile_160402.txt',
#          'turnstile_160409.txt',
#          'turnstile_160416.txt',
#          'turnstile_160423.txt',
#          'turnstile_160430.txt',
#          'turnstile_160507.txt',
#          'turnstile_160514.txt',
#          'turnstile_160521.txt',
#          'turnstile_160528.txt',
#          'turnstile_160604.txt',
#          'turnstile_160611.txt',
#          'turnstile_160618.txt',
#          'turnstile_160625.txt']

# frames = []
# for filename in glob.glob("turnstile_16*.txt"): # glob() finds every file in the directory matching a pattern
#     frames.append(pd.read_csv(filename))   # make list of dataframes
# df = pd.concat(frames)   # concatenate all dataframes

files = ['turnstile_160507.txt',
         'turnstile_160514.txt',
         'turnstile_160521.txt',
         'turnstile_160528.txt']


mta = pd.DataFrame()
frames = []

for file in files:
    df = pd.read_csv(file)
    frames.append(df)

mta = pd.concat(frames)

In [16]:
mta.columns = [n.lower().strip() for n in mta.columns]

mta['date_time'] = mta['date'] + " " + mta['time']
mta.date_time = pd.to_datetime(mta.date_time, format='%m/%d/%Y %H:%M:%S')

# df['dt'] = df.date.str.cat(df.time, sep=' ')
# df.dt = df.dt.apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y %H:%M:%S'))

In [103]:
mta.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 777903 entries, 0 to 114014
Data columns (total 15 columns):
c/a             777903 non-null object
unit            777903 non-null object
scp             777903 non-null object
station         777903 non-null object
linename        777903 non-null object
division        777903 non-null object
date            777903 non-null object
time            777903 non-null object
desc            777903 non-null object
entries         777903 non-null int64
exits           777903 non-null int64
date_time       777903 non-null datetime64[ns]
entries_diff    773314 non-null float64
exits_diff      773314 non-null float64
traffic         773314 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(2), object(9)
memory usage: 95.0+ MB


In [18]:
print mta.groupby(['c/a', 'unit', 'scp', 'station', 'linename', 'date_time']).size().sort_values(ascending=False)


c/a   unit  scp       station          linename     date_time          
N418  R269  01-05-00  BEDFORD-NOSTRAN  G            2016-05-19 20:00:00    2
                                                    2016-05-14 16:00:00    2
                                                    2016-05-15 00:00:00    2
                                                    2016-05-19 00:00:00    2
                                                    2016-05-15 08:00:00    2
                                                    2016-05-15 12:00:00    2
                                                    2016-05-15 16:00:00    2
                                                    2016-05-15 20:00:00    2
                                                    2016-05-16 00:00:00    2
                                                    2016-05-18 20:00:00    2
                                                    2016-05-16 08:00:00    2
                                                    2016-05-16 12:00:00    2
    

In [8]:
#mta.groupby(['c/a', 'unit', 'scp', 'station', 'linename', 'date_time']).filter(lambda x: len(x)>=2)

In [19]:
mta.drop_duplicates(subset=['c/a', 'unit', 'scp', 'station', 'linename', 'date_time'], inplace=True)

In [20]:
mta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 777903 entries, 0 to 195728
Data columns (total 12 columns):
c/a          777903 non-null object
unit         777903 non-null object
scp          777903 non-null object
station      777903 non-null object
linename     777903 non-null object
division     777903 non-null object
date         777903 non-null object
time         777903 non-null object
desc         777903 non-null object
entries      777903 non-null int64
exits        777903 non-null int64
date_time    777903 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 77.2+ MB


In [ ]:
mta.sort_values('date_time', inplace=True)

In [60]:
mta['entries_diff'] = mta.groupby(['c/a', 'unit', 'scp', 'station', 'linename']).entries. \
    transform(lambda x: abs(x-x.shift()))
mta['exits_diff'] = mta.groupby(['c/a', 'unit', 'scp', 'station', 'linename']).exits. \
    transform(lambda x: abs(x-x.shift()))
print mta['entries_diff'].value_counts(dropna=False)

 0.0            111441
 1.0             13363
 2.0              9175
 3.0              8145
 4.0              7441
 5.0              7080
 6.0              6731
 7.0              6377
 8.0              6124
 9.0              6082
 10.0             5777
 11.0             5596
 12.0             5412
 13.0             5310
 14.0             5155
 15.0             4985
 16.0             4780
 17.0             4644
NaN               4589
 18.0             4495
 19.0             4476
 20.0             4358
 21.0             4148
 22.0             3968
 23.0             3885
 24.0             3821
 25.0             3616
 26.0             3572
 27.0             3535
 28.0             3440
                 ...  
 2394.0              1
 2400.0              1
 2489.0              1
 2488.0              1
 2487.0              1
 2485.0              1
 2474.0              1
 2473.0              1
 2472.0              1
 2471.0              1
 2465.0              1
 2463.0              1
 2462.0    

In [61]:
#print mta.loc[mta['entries_diff'] < 0, 'entries_diff']
print mta[mta['station'] == 'CHAMBERS ST'].sort_values(['c/a', 'unit', 'scp', 'station', 'linename']) \
    [['scp', 'station','date_time', 'entries', 'entries_diff']]

             scp      station           date_time  entries  entries_diff
11458   01-00-00  CHAMBERS ST 2016-04-30 00:00:00  2888901           NaN
11459   01-00-00  CHAMBERS ST 2016-04-30 04:00:00  2888904           3.0
11460   01-00-00  CHAMBERS ST 2016-04-30 08:00:00  2888909           5.0
11461   01-00-00  CHAMBERS ST 2016-04-30 12:00:00  2888927          18.0
11462   01-00-00  CHAMBERS ST 2016-04-30 16:00:00  2888965          38.0
11463   01-00-00  CHAMBERS ST 2016-04-30 20:00:00  2889005          40.0
11464   01-00-00  CHAMBERS ST 2016-05-01 00:00:00  2889022          17.0
11465   01-00-00  CHAMBERS ST 2016-05-01 04:00:00  2889022           0.0
11466   01-00-00  CHAMBERS ST 2016-05-01 08:00:00  2889023           1.0
11467   01-00-00  CHAMBERS ST 2016-05-01 12:00:00  2889038          15.0
11468   01-00-00  CHAMBERS ST 2016-05-01 16:00:00  2889069          31.0
11469   01-00-00  CHAMBERS ST 2016-05-01 20:00:00  2889088          19.0
11470   01-00-00  CHAMBERS ST 2016-05-02 00:00:00  

In [ ]:
mta['traffic'] = mta.entries_diff + mta.exits_diff

In [102]:
test = mta.groupby(mta.traffic > 5000).size()
test/test.sum()

#mta.sort_values(['c/a', 'unit', 'scp', 'station', 'linename']). \
 #   groupby(['c/a', 'unit', 'scp', 'station', 'linename']).filter(lambda x: x['traffic'].max() > 5000)
pd.options.display.max_rows = None

#Find giant stupid diff value at Canal St
#Cut at 3,000 per turnstile
mta.sort_values(['c/a', 'unit', 'scp', 'station', 'linename']). \
    groupby(['c/a', 'unit', 'scp', 'station', 'linename']). \
    get_group(('A066', 'R118', '00-00-00', 'CANAL ST', 'JNQRZ6'))[:]

,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits,date_time,entries_diff,exits_diff,traffic
11332,A066,R118,00-00-00,CANAL ST,JNQRZ6,BMT,04/30/2016,01:00:00,REGULAR,9986785,6886280,2016-04-30 01:00:00,NaN,NaN,NaN
11333,A066,R118,00-00-00,CANAL ST,JNQRZ6,BMT,04/30/2016,05:00:00,REGULAR,9986852,6886314,2016-04-30 05:00:00,6.700000e+01,3.400000e+01,1.010000e+02
11334,A066,R118,00-00-00,CANAL ST,JNQRZ6,BMT,04/30/2016,09:00:00,REGULAR,9986948,6886456,2016-04-30 09:00:00,9.600000e+01,1.420000e+02,2.380000e+02
11335,A066,R118,00-00-00,CANAL ST,JNQRZ6,BMT,04/30/2016,13:00:00,REGULAR,9987376,6886701,2016-04-30 13:00:00,4.280000e+02,2.450000e+02,6.730000e+02
11336,A066,R118,00-00-00,CANAL ST,JNQRZ6,BMT,04/30/2016,17:00:00,REGULAR,9988530,6886915,2016-04-30 17:00:00,1.154000e+03,2.140000e+02,1.368000e+03
11337,A066,R118,00-00-00,CANAL ST,JNQRZ6,BMT,04/30/2016,21:00:00,REGULAR,9989901,6887168,2016-04-30 21:00:00,1.371000e+03,2.530000e+02,1.624000e+03
11338,A066,R118,00-00-00,CANAL ST,JNQRZ6,BMT,05/01/2016,01:00:00,REGULAR,9990529,6887258,2016-05-01 01:00:00,6.280000e+02,9.000000e+01,7.180000e+02
11339,A066,R118,00-00-00,CANAL ST,JNQRZ6,BMT,05/01/2016,05:00:00,REGULAR,9990579,6887279,2016-05-01 05:00:00,5.000000e+01,2.100000e+01,7.100000e+01
11340,A066,R118,00-00-00,CANAL ST,JNQRZ6,BMT,05/01/2016,09:00:00,REGULAR,9990662,6887395,2016-05-01 09:00:00,8.300000e+01,1.160000e+02,1.990000e+02
11341,A066,R118,00-00-00,CANAL ST,JNQRZ6,BMT,05/01/2016,13:00:00,REGULAR,9991070,6887658,2016-05-01 13:00:00,4.080000e+02,2.630000e+02,6.710000e+02


In [92]:
print mta.traffic.dropna().describe()
# plt.hist(mta.traffic[mta.traffic < 4000])
# mta.boxplot('traffic')

count    7.733140e+05
mean     3.427747e+04
std      9.370308e+06
min      0.000000e+00
25%      3.300000e+01
50%      1.880000e+02
75%      4.940000e+02
max      3.929731e+09
Name: traffic, dtype: float64


In [49]:
mta_summary = mta.sort_values('date_time').\
              groupby(['station', 'linename', 'date_time'], as_index=False)\
              [['entries', 'exits']].sum()

In [48]:
mta_summary['entires_real'] = mta_summary['entries'].diff()
mta_summary['exits_real'] = mta_summary['exits'].diff()


In [47]:
good_time = mta_summary.date_time.apply(lambda x: x.hour >= 11 and x.hour <=19)
mta_timeframe = mta_summary[good_time]
mta_timeframe

,station,linename,date_time,entries,exits,entires_real,exits_real
3,1 AV,L,2016-03-26 12:00:00,1560079106,3475276304,2779.0,1963.0
4,1 AV,L,2016-03-26 16:00:00,1560083544,3475279125,4438.0,2821.0
9,1 AV,L,2016-03-27 12:00:00,1560096866,3475288034,2106.0,1545.0
10,1 AV,L,2016-03-27 16:00:00,1560100368,3475290451,3502.0,2417.0
15,1 AV,L,2016-03-28 12:00:00,1560114343,3475301335,5755.0,4031.0
16,1 AV,L,2016-03-28 16:00:00,1560119331,3475303835,4988.0,2500.0
21,1 AV,L,2016-03-29 12:00:00,1560137663,3475315715,5783.0,3947.0
22,1 AV,L,2016-03-29 16:00:00,1560142720,3475318336,5057.0,2621.0
27,1 AV,L,2016-03-30 12:00:00,1560162142,3475331008,5531.0,4171.0
28,1 AV,L,2016-03-30 16:00:00,1560167314,3475333553,5172.0,2545.0


In [134]:
inc = pd.read_csv('income_by_subway_stops.csv')
inc.columns = [n.lower().replace(' ', '_') for n in inc.columns]

In [135]:
inc = inc.dropna(subset=['stop_name'])
inc2 = inc2.groupby(['lat','long']).head(1)
inc2['station_line'] = inc2.stop_name.str.upper() + ' ' + inc2.subway_line

In [137]:
inc2.head(25)

(428, 12)

In [107]:
mta['join_station_line'] = mta.station + ' ' + mta.linename
mta[['station', 'linename', 'join_station_line']].head(25)

,station,linename,join_station_line
0,59 ST,NQR456,59 ST NQR456
57457,CHAMBERS ST,ACE23,CHAMBERS ST ACE23
6363,PRINCE ST,NR,PRINCE ST NR
57415,CHAMBERS ST,ACE23,CHAMBERS ST ACE23
57373,CHAMBERS ST,ACE23,CHAMBERS ST ACE23
57332,CHAMBERS ST,ACE23,CHAMBERS ST ACE23
57291,CHAMBERS ST,ACE23,CHAMBERS ST ACE23
6405,PRINCE ST,NR,PRINCE ST NR
57250,CHAMBERS ST,ACE23,CHAMBERS ST ACE23
185230,ATL AV-BARCLAY,2345BDNQR,ATL AV-BARCLAY 2345BDNQR


In [110]:
station_line_list = mta['join_station_line'].unique()

In [152]:
inc2['join_station_line'] = inc2.station_line.apply(lambda x: \
                                        difflib.get_close_matches(x, station_line_list, n=1, cutoff=.3))

In [157]:
inc2.sort_values('join_station_line', na_position='first')
inc2[inc2.stop_name.str.contains('34 ')]

,subway_line,lat,long,county_name,position,income2011,income2000,stop_name,county,tract,tract_lookup,station_line,join_station_line
12,1,40.751020,-73.990323,Manhattan,12.0,155278.0,71641.0,34 St.-Penn Station,61.0,10900.0,10900.0,34 ST.-PENN STATION 1,[34 ST-PENN STA 123]
269,A,40.751996,-73.993327,Manhattan,26.0,51827.0,66786.0,34 St.-Penn Station,61.0,10100.0,10100.0,34 ST.-PENN STATION A,[34 ST-PENN STA ACE]
292,B,40.749891,-73.987495,Manhattan,13.0,76659.0,41276.0,34 St.-Herald Square,61.0,7600.0,7600.0,34 ST.-HERALD SQUARE B,[34 ST-HERALD SQ BDFMNQR]


In [ ]:
inc2.join_station_line = inc2.join_station_line.apply(lambda x: x[0])

In [163]:
inc2_for_merge = inc2[['lat', 'long', 'county_name', 'income2011', 'join_station_line']]

In [161]:
mta_merge = pd.merge(mta, inc2_for_merge, how='left', on='join_station_line')

,subway_line,lat,long,county_name,position,income2011,income2000,stop_name,county,tract,tract_lookup,station_line,join_station_line
1,1,40.701961,-74.013397,Manhattan,1.0,124048.0,105456.0,South Ferry,61.0,900.0,900.0,SOUTH FERRY 1,SOUTH FERRY 1R
2,1,40.708107,-74.013445,Manhattan,2.0,114412.0,81609.0,Rector St.,61.0,1300.0,1300.0,RECTOR ST. 1,RECTOR ST 1
3,1,40.715576,-74.009031,Manhattan,3.0,205192.0,128384.0,Chambers St.,61.0,2100.0,2100.0,CHAMBERS ST. 1,CHAMBERS ST 123
4,1,40.719918,-74.006620,Manhattan,4.0,164656.0,113332.0,Franklin St.,61.0,3300.0,3300.0,FRANKLIN ST. 1,FRANKLIN ST 1
5,1,40.722497,-74.005994,Manhattan,5.0,75556.0,85483.0,Canal St.,61.0,3700.0,3700.0,CANAL ST. 1,CANAL ST 1
6,1,40.728656,-74.005086,Manhattan,6.0,74706.0,57567.0,Houston St.,61.0,6700.0,6700.0,HOUSTON ST. 1,HOUSTON ST 1
7,1,40.733402,-74.002748,Manhattan,7.0,104857.0,58202.0,Christopher St.-Sheridan Square,61.0,7300.0,7300.0,CHRISTOPHER ST.-SHERIDAN SQUARE 1,CHRISTOPHER ST 1
8,1,40.737826,-74.000201,Manhattan,8.0,94292.0,81696.0,14 St.-7 Ave.,61.0,8100.0,8100.0,14 ST.-7 AVE. 1,14 ST ACEL
9,1,40.741617,-73.997197,Manhattan,9.0,110642.0,56939.0,18 St.,61.0,8700.0,8700.0,18 ST. 1,18 ST 1
10,1,40.744120,-73.995348,Manhattan,10.0,114655.0,58381.0,23 St.,61.0,9100.0,9100.0,23 ST. 1,23 ST 1
